<a href="https://colab.research.google.com/github/xiaoya27/Aerial-Image-Recognition/blob/main/Resnet_Aerial_Image_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aerial imagery Project

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Step1 : Install Requirements 


In [3]:
!pip install neptune-client==0.4.130

import neptune
from google.colab import output
output.clear() 

In [4]:
%cd /content/drive/MyDrive/workingfolder/2021/starting_kit/

/content/drive/MyDrive/workingfolder/2021/starting_kit


In [5]:
neptune.init(
    api_token="xxx",
    project_qualified_name="xiaoya27/AerialImageClassification"
)

Project(xiaoya27/AerialImageClassification)

In [6]:
import numpy as np
import random
import re
from matplotlib import pyplot as plt

In [7]:
scene_class = ['beach', 'chaparral','cloud','desert','forest','island','lake','meadow','mountain','river','sea','snowberg','wetland']
model_dir = "sample_code_submission"
result_dir = 'sample_result_submission/' 
problem_dir = 'ingestion_program/'  
score_dir = 'scoring_program/'
from sys import path; path.append(model_dir); path.append(problem_dir); path.append(score_dir)

In [8]:
def data_loading():
  #data_dir = 'sample_data'
  data_dir = '../public_data' # download "public_data" from the challenge website
  data_name = 'Areal'
  from data_manager import DataManager
  D = DataManager(data_name, data_dir, replace_missing=False, verbose=True)
  print(D)
  X_train = D.data['X_train']
  Y_train = D.data['Y_train']
  X_valid = D.data['X_valid']
  X_test = D.data['X_test']
  with open('data.npy', 'wb') as f:
      np.save(f, X_train)
      np.save(f, Y_train)
      np.save(f, X_valid )
      np.save(f, X_test)

In [9]:
# run it once
#data_loading()

In [10]:
with open('data.npy', 'rb') as f:
    X_train = np.load(f)
    Y_train = np.load(f)
    X_valid = np.load(f)
    X_test = np.load(f)

In [11]:
Y_train.shape

(5200, 1)

In [12]:
val_percent = 0.05
train_size=int((1-val_percent)*len(X_train))

In [13]:
#data_dir = 'sample_data'
data_dir = '../public_data' # download "public_data" from the challenge website
data_name = 'Areal'

In [14]:

params={}
params['batch_size']=16
params['val_percent']=val_percent



### Step 2 : Building training dataset

In [15]:
#X_train.reshape((5200,128,128,3))

%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

from tensorflow.keras.applications import ResNet50 as ResNet

feature_extractor_name='ResNet'

Tensorflow version 2.4.0


In [16]:
# check AUTOTUNE in https://www.tensorflow.org/guide/data_performance

In [17]:
#View images from the dataset

def view_image(ds):
    image, label = next(iter(ds)) # extract 1 batch from the dataset
    image = image.numpy()
    label = label.numpy()

    fig = plt.figure(figsize=(22,11))
    for i in range(8):
        ax = fig.add_subplot(2, 4, i+1, xticks=[], yticks=[])
        ax.imshow(image[i])
        ax.set_title(f"Label: {label[i]}")

In [18]:
#view_image(dataset)

In [19]:
def process_image(image, label):
    # cast and normalize image
    image = tf.reshape(image, (128,128,3))
    image = tf.image.resize(image, [224, 224],method='nearest')
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.random_flip_left_right(image)
    image = tf.keras.applications.resnet.preprocess_input(image)
    return image, label


dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(buffer_size=1024)


In [20]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

train_dataset = dataset.map(process_image).batch(params['batch_size']).cache().prefetch(tf.data.experimental.AUTOTUNE)

val_dataset = dataset.map(process_image).batch(params['batch_size']).cache().prefetch(tf.data.experimental.AUTOTUNE)

In [21]:
from tensorflow.keras.callbacks import Callback
class NeptuneLogger(Callback):

    def on_batch_end(self, batch, logs={}):
        for log_name, log_value in logs.items():
            neptune.log_metric(f'batch_{log_name}', log_value)

    def on_epoch_end(self, epoch, logs={}):
        for log_name, log_value in logs.items():
            neptune.log_metric(f'epoch_{log_name}', log_value)

neptune.create_experiment(name='Aerial-image',
                          params=params,
                          tags=['Tensorflow',feature_extractor_name],
                          )

https://ui.neptune.ai/xiaoya27/AerialImageClassification/e/AER-6


Experiment(AER-6)

### Step 3 . Building&training model

In [22]:
feature_extractor = ResNet(input_shape=(224,224,3),
                                          include_top=False,
                                          weights='imagenet')
# what is input shape of backend model
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
                                          layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                                          layers.experimental.preprocessing.RandomRotation(0.2),])
input = tf.keras.Input(shape=(224,224,3))
x = data_augmentation(input)
x = feature_extractor(input,training=True)

x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dense(512, activation="relu")(x)
x = tf.keras.layers.Dense(13, activation="softmax", name="classification")(x)                                  

checkpoint_path = "training_res_1/cp.ckpt"

94773248/94765736 [==============================] - 1s 0us/step


In [23]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

with tf.device('/device:GPU:0'):
  model = tf.keras.Model(inputs=input, outputs=x)

  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy', run_eagerly=True)
  model.fit(train_dataset,
            epochs=15,
            validation_data=val_dataset,
            callbacks=[cp_callback,NeptuneLogger()]
            )
# TBC https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/transfer_learning.ipynb#scrollTo=DgzQX6Veb2WT

Epoch 1/15
325/325 [==============================] - 114s 329ms/step - loss: 2.1354 - accuracy: 0.2796 - val_loss: 1.5990 - val_accuracy: 0.4333

Epoch 00001: saving model to training_res_1/cp.ckpt
Epoch 2/15
325/325 [==============================] - 96s 295ms/step - loss: 1.4782 - accuracy: 0.4754 - val_loss: 1.2916 - val_accuracy: 0.5556

Epoch 00002: saving model to training_res_1/cp.ckpt
Epoch 3/15
325/325 [==============================] - 95s 294ms/step - loss: 1.1420 - accuracy: 0.6206 - val_loss: 1.0208 - val_accuracy: 0.6552

Epoch 00003: saving model to training_res_1/cp.ckpt
Epoch 4/15
325/325 [==============================] - 95s 294ms/step - loss: 0.9755 - accuracy: 0.6775 - val_loss: 0.8231 - val_accuracy: 0.7267

Epoch 00004: saving model to training_res_1/cp.ckpt
Epoch 5/15
325/325 [==============================] - 95s 294ms/step - loss: 0.8171 - accuracy: 0.7166 - val_loss: 0.7860 - val_accuracy: 0.7229

Epoch 00005: saving model to training_res_1/cp.ckpt
Epoch 6/1

In [24]:
neptune.stop()